In [1]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pickle
import pandas as pd
from datetime import date, timedelta
import numpy as np
import investpy
import yfinance as yf
import eikon as ek

In [2]:
def valid_dates(dates_all):
    dates = []
    i = 0
    while True:
        dates.append(dates_all[i])
        if dates_all[i] > pd.to_datetime(date.today()):
            break
        i = i + 1
    return dates

In [13]:
def prepare_portfolio_data(tickers,api,country):
    data = pd.DataFrame()
    for ticker in tickers:
        try:
            temp_og = get_data(ticker, api,country)
            data = pd.concat([data, temp_og["Close"].to_frame().astype(float).rename(columns={"Close":ticker}).set_index(temp_og["Date"])], axis=1)
        except:
            print(f"{ticker} not processed")
    data.reset_index(inplace=True)
    return data

In [14]:
def get_data_investpy( symbol, country, from_date, to_date ):
    find = investpy.search.search_quotes(text=symbol, products=["stocks", "etfs", "indices", "currencies"])
    for f in find:
        #print( f )
        if f.symbol.lower() == symbol.lower() and f.country.lower() == country.lower():
            break
    if f.symbol.lower() != symbol.lower():
        return None
    ret = f.retrieve_historical_data(from_date=from_date, to_date=to_date)
    if ret is None:
        try:
            ret = investpy.get_stock_historical_data(stock=symbol,
                                                     country=country,
                                                     from_date=from_date,
                                                     to_date=to_date)
        except:
            ret = None
    if ret is None:
        try:
            ret = investpy.get_etf_historical_data(etf=symbol,
                                                   country=country,
                                                   from_date=from_date,
                                                   to_date=to_date)
        except:
            ret = None
    if ret is None:
        try:
            ret = investpy.get_index_historical_data(index=symbol,
                                                     country=country,
                                                     from_date=from_date,
                                                     to_date=to_date)
        except:
            ret = None

    if ret is None:
        try:
            ret = investpy.currency_crosses.get_currency_cross_historical_data(currency_cross=symbol,
                                                                               from_date=from_date,
                                                                               to_date=to_date)
        except:
            ret = None
    ret.drop(["Change Pct"], axis=1, inplace=True)
    return ret

def get_data(ticker, api,country):

    if api == "yfinance":

        temp_og = yf.download(ticker, start = '2007-01-01', end= str(date.today()+timedelta(1)))
        if len(temp_og)==0:
            temp_og = yf.download(ticker, start='2007-01-01', end=str(date.today()))
        temp_og.reset_index(inplace=True)
        temp_og.drop(['Adj Close'], axis=1, inplace=True)
        if ticker=="GOLDBEES.NS":
            temp_og = temp_og.loc[temp_og["Close"]>1]

    if api =="investpy":
        temp_og = get_data_investpy(symbol=ticker, country=country, from_date="01/01/2007",to_date=(date.today()+timedelta(1)).strftime("%d/%m/%Y"))
        temp_og.reset_index(inplace=True)

    if api == "reuters":
        temp_og = ek.get_timeseries(ticker, start_date='2007-01-01', end_date=str(date.today() + timedelta(1)))
        temp_og.reset_index(inplace=True)
        temp_og.rename(columns={"HIGH": "High", "CLOSE": "Close", "LOW": "Low", "OPEN": "Open", "VOLUME": "Volume"},
                       inplace=True)
        temp_og.drop(['COUNT'], axis=1, inplace=True)

    return temp_og

In [15]:

index = ".BSESN"
with open(f'BSE_Constituents.pkl', 'rb') as file:
    constituents = pickle.load(file)
constituents_all = []
for i in range(len(constituents)):
    constituents_all = constituents_all + constituents.iloc[i][0]
tickers = list(set(constituents_all))
tickers = [ticker[:-3] for ticker in tickers]

In [16]:
data_inp = prepare_portfolio_data(tickers, "investpy", "india")

In [18]:
data_inp = data_inp[data_inp['Date'] != '2020-11-14']
data_inp = data_inp[data_inp['Date'] != '2020-11-17']